In [ ]:
print()

In [ ]:
%env CUDA_VISIBLE_DEVICES=1

In [ ]:
from experiments.architecture import *

In [ ]:
fetch('output/experiments/architecture/results/', 'output/experiments/architecture/results/')
raw = load_opt_results('conv-opt')

In [ ]:
plot_results(raw)

In [ ]:
time = raw.xs('time', 1, 4).pipe(lambda df: df - df.iloc[0]).max()
train = raw.xs('train', 1, 4).ewm(span=100).mean().min()
steps = raw.xs('train', 1, 4).pipe(lambda df: df.ffill().where(df.bfill().notnull())).notnull().sum()
picks = train.groupby(['n', 'l']).idxmin()

aug = (train
        .groupby(['n', 'l']).min()
        .reset_index())
aug.columns = ['width', 'depth', 'rv']

aug['time'] = time[picks].values
aug['steps'] = steps[picks].values

aug = (aug
        .assign(params=lambda df: (df.width**2 + df.width)*df.depth)
        .assign(memory=lambda df: df.width*df.depth)
        .assign(flops=lambda df: (df.width**3 + df.width)*df.depth))

aug['total_flops'] = aug.flops*aug.steps

In [ ]:
ax = plot_envelope(aug, 'depth')
ax.set_title((
        'performance envelopes for predicting outcome of 11x11 Hex games with convolutional networks\n'
        '(width doubles line-to-line, depth doubles node-to-node)'))
ax.legend()

In [ ]:
raw.xs('train', 1, 4).ewm(span=100).mean()[32, 4].plot()

In [ ]:
aug